In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict
from dotenv import load_dotenv

In [3]:
load_dotenv()

model = ChatOpenAI(model="gpt-5-nano")

In [4]:
class BlogState(TypedDict):

    title: str
    outline: str
    content: str

In [5]:
def create_outline(state: BlogState) -> BlogState:

    # fetch title
    title = state['title']

    # call llm gen outline
    prompt = f'Generate a detailed outline for a blog on the topic - {title}'
    outline = model.invoke(prompt).content

    # update state
    state['outline'] = outline

    return state

In [6]:
def create_blog(state: BlogState) -> BlogState:

    title = state['title']
    outline = state['outline']

    prompt = f'Write a detailed blog on the title - {title} using the follwing outline \n {outline}'

    content = model.invoke(prompt).content

    state['content'] = content

    return state

In [7]:
graph = StateGraph(BlogState)

# nodes
graph.add_node('create_outline', create_outline)
graph.add_node('create_blog', create_blog)

# edges
graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog', END)

workflow = graph.compile()

In [8]:
intial_state = {'title': 'Rise of AI in India'}

final_state = workflow.invoke(intial_state)

print(final_state)

{'title': 'Rise of AI in India', 'outline': 'Here’s a detailed, ready-to-use outline for a blog post (or a multi-post series) on the Rise of AI in India. It covers context, current landscape, sectoral deep dives, policy, ethics, challenges, and practical takeaways.\n\nProposed title options\n- Rise of AI in India: Driving Transformation Across Sectors and Society\n- India and the AI Era: Growth, Policy, and the Path Ahead\n- From Demographics to Digital Transformation: India’s AI Journey\n\nTarget audience\n- Policymakers and public-sector officials\n- Startup founders, entrepreneurs, and investors\n- CIOs/CTOs and business leaders in large enterprises\n- AI researchers, students, and industry analysts\n- Rural and SME stakeholders exploring AI adoption\n\nTone and style\n- Informative, data-driven, accessible to non-experts\n- Use real-world case studies and concrete examples\n- Provide practical takeaways and clear next steps\n\nSEO and meta plan (optional to fill in later)\n- Primar